In [1]:
from Utils.writeSubmission import write_submission
import pandas as pd
import numpy as np
import scipy.sparse as sps

In [2]:
dataset = pd.read_csv('../Input/URM.csv')
dataset

KeyboardInterrupt: 

In [20]:
dataset.drop(dataset[dataset.Data != 0].index, inplace=True)
dataset

,UserID,ItemID,Impressions,Data
1,0,21,NaN,0
2,0,21,NaN,0
3,0,21,"20,21,22,23,24,25,26,27,28,29",0
37,0,124,"124,20,30,22,23,348,25,206,386,631,32,56",0
42,0,808,NaN,0
...,...,...,...,...
5826499,41628,15971,NaN,0
5826500,41628,19992,NaN,0
5826501,41628,20448,NaN,0
5826504,41628,22882,NaN,0


In [21]:
dataset['Data'] = dataset['Data'].replace(0, 1)
dataset

,UserID,ItemID,Impressions,Data
1,0,21,NaN,1
2,0,21,NaN,1
3,0,21,"20,21,22,23,24,25,26,27,28,29",1
37,0,124,"124,20,30,22,23,348,25,206,386,631,32,56",1
42,0,808,NaN,1
...,...,...,...,...
5826499,41628,15971,NaN,1
5826500,41628,19992,NaN,1
5826501,41628,20448,NaN,1
5826504,41628,22882,NaN,1


In [22]:
URM = sps.coo_matrix((dataset["Data"].values,
                          (dataset["UserID"].values, dataset["ItemID"].values)))

In [23]:
URM = URM.tocsr()

In [24]:
class TopPopRecommender(object):

    def fit(self, URM_train):

        self.URM_train = URM_train

        item_popularity = np.ediff1d(URM_train.tocsc().indptr)

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popular_items = np.argsort(item_popularity)
        self.popular_items = np.flip(self.popular_items, axis=0)

    def recommend(self, user_id, at=20, remove_seen=True):

        if remove_seen:
            seen_items = self.URM_train.indices[self.URM_train.indptr[user_id]:self.URM_train.indptr[user_id + 1]]

            unseen_items_mask = np.in1d(self.popular_items, seen_items, assume_unique=True, invert=True)

            unseen_items = self.popular_items[unseen_items_mask]

            recommended_items = unseen_items[0:at]

        else:
            recommended_items = self.popular_items[0:at]

        return recommended_items

In [25]:
topPopRecommender_removeSeen = TopPopRecommender()
topPopRecommender_removeSeen.fit(URM)

In [26]:
write_submission(recommender=topPopRecommender_removeSeen, target_users_path="../Input/data_target_users_test.csv",
                     out_path='../Output/{}_submission.csv'.format('TopPop'))